# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [3]:
data

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.20,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.60,13408,United Kingdom
...,...,...,...,...,...,...,...,...,...
396029,580691,2011-12-05 15:48:00,90214W,"LETTER ""W"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396030,580691,2011-12-05 15:48:00,90214Z,"LETTER ""Z"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396031,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom
396032,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year 
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
#data.groupby('Country')[['Quantity','Revenue']].sum()[:-1]
#data[(data['InvoiceDate'].dt.month==4)&(data['InvoiceDate'].dt.year==2011)]

In [5]:
yearData = data[(data['Year']==2011)]
data1 = yearData[(yearData['Month']==4)]
dataGroup = data1.groupby('Country')[['Quantity','Revenue']].sum()[:-1]
dataGroup

,Quantity,Revenue
Country,,
Australia,224,421.60
Austria,308,584.78
Belgium,1170,1788.48
Brazil,356,1143.60
Channel Islands,96,243.00
EIRE,4129,7270.50
Finland,810,1368.92
France,2265,3899.31
Germany,5702,10994.79


In [6]:
data1Bar = dataGroup.iplot(kind='bar')
data1Bar

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
data2 = data[data['Country']=='France'][(data['InvoiceDate'].dt.month<6)&(data['InvoiceDate'].dt.year==2011)]
dataG2 = data2.groupby('Month')[['Quantity','Revenue']].sum()
data2Line = dataG2.iplot(kind='line')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [8]:
data3 = data[data['Description']=='PARTY BUNTING']
data3SC = data3.groupby('Country', as_index=False).agg({'Quantity':'mean','UnitPrice':'mean'})

In [9]:
data3SC.iplot(x='Quantity', y='UnitPrice', categories='Country',
           xTitle='Quantity', yTitle='Unit Price')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [10]:
countrys = ['EIRE','Germany','France','Netherlands']
data4 = data[data['Country'].isin(countrys)][['Country','Quantity','InvoiceNo']]
data4P = data4.pivot_table(values='Quantity', columns='Country', 
                      index='InvoiceNo', aggfunc='sum')
data4P.iplot(subplots=True, kind='hist', xTitle='InvoiceNo', 
           yTitle='Quantity')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [16]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [17]:
data5_1 = data[data['Country'].isin(country_list)]
data5 = data5_1[data5_1['Description'].isin(product_list)]

In [18]:
data5p = data5.pivot_table(values='Revenue', columns='Country', 
                      index='Description', aggfunc='sum')
data5p

Country,EIRE,France,Germany,Netherlands
Description,,,,
CREAM HANGING HEART T-LIGHT HOLDER,2740.80,131.75,35.40,1167.00
JUMBO BAG RED RETROSPOT,278.72,903.37,1072.76,3468.00
REGENCY CAKESTAND 3 TIER,7388.55,2816.85,9061.95,3166.35


In [19]:
data5p=data5p.reset_index()
data5p.iplot(kind='bar', x='Description', xTitle='Products', 
         yTitle='Revenue')


## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.